# Diffusion training in Fourier domain

In [1]:
from torch.optim.lr_scheduler import _LRScheduler

class GradualWarmupScheduler(_LRScheduler):
    def __init__(self, optimizer, multiplier, warm_epoch, after_scheduler=None):
        self.multiplier = multiplier
        self.total_epoch = warm_epoch
        self.after_scheduler = after_scheduler
        self.finished = False
        self.last_epoch = None
        self.base_lrs = None
        super().__init__(optimizer)

    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]


    def step(self, epoch=None, metrics=None):
        if self.finished and self.after_scheduler:
            if epoch is None:
                self.after_scheduler.step(None)
            else:
                self.after_scheduler.step(epoch - self.total_epoch)
        else:
            return super(GradualWarmupScheduler, self).step(epoch)

In [1]:
# ModelCondition.py

import math
from telnetlib import PRAGMA_HEARTBEAT
import torch
from torch import nn
from torch.nn import init
from torch.nn import functional as F


def drop_connect(x, drop_ratio):
    keep_ratio = 1.0 - drop_ratio
    mask = torch.empty([x.shape[0], 1, 1, 1], dtype=x.dtype, device=x.device)
    mask.bernoulli_(p=keep_ratio)
    x.div_(keep_ratio)
    x.mul_(mask)
    return x

class Swish(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)


class TimeEmbedding(nn.Module):
    def __init__(self, T, d_model, dim):
        assert d_model % 2 == 0
        super().__init__()
        emb = torch.arange(0, d_model, step=2) / d_model * math.log(10000)
        emb = torch.exp(-emb)
        pos = torch.arange(T).float()
        emb = pos[:, None] * emb[None, :]
        assert list(emb.shape) == [T, d_model // 2]
        emb = torch.stack([torch.sin(emb), torch.cos(emb)], dim=-1)
        assert list(emb.shape) == [T, d_model // 2, 2]
        emb = emb.view(T, d_model)

        self.timembedding = nn.Sequential(
            nn.Embedding.from_pretrained(emb, freeze=False),
            nn.Linear(d_model, dim),
            Swish(),
            nn.Linear(dim, dim),
        )

    def forward(self, t):
        emb = self.timembedding(t)
        return emb


class ConditionalEmbedding(nn.Module):
    def __init__(self, num_labels, d_model, dim):
        assert d_model % 2 == 0
        super().__init__()
        self.condEmbedding = nn.Sequential(
            nn.Embedding(num_embeddings=num_labels + 1, embedding_dim=d_model, padding_idx=0),
            nn.Linear(d_model, dim),
            Swish(),
            nn.Linear(dim, dim),
        )

    def forward(self, t):
        emb = self.condEmbedding(t)
        return emb


class DownSample(nn.Module):
    def __init__(self, in_ch):
        super().__init__()
        self.c1 = nn.Conv2d(in_ch, in_ch, 3, stride=2, padding=1)
        self.c2 = nn.Conv2d(in_ch, in_ch, 5, stride=2, padding=2)

    def forward(self, x, temb, cemb):
        x = self.c1(x) + self.c2(x)
        return x


class UpSample(nn.Module):
    def __init__(self, in_ch):
        super().__init__()
        self.c = nn.Conv2d(in_ch, in_ch, 3, stride=1, padding=1)
        self.t = nn.ConvTranspose2d(in_ch, in_ch, 5, 2, 2, 1)

    def forward(self, x, temb, cemb):
        _, _, H, W = x.shape
        x = self.t(x)
        x = self.c(x)
        return x


class AttnBlock(nn.Module):
    def __init__(self, in_ch):
        super().__init__()
        self.group_norm = nn.GroupNorm(24, in_ch)
        self.proj_q = nn.Conv2d(in_ch, in_ch, 1, stride=1, padding=0)
        self.proj_k = nn.Conv2d(in_ch, in_ch, 1, stride=1, padding=0)
        self.proj_v = nn.Conv2d(in_ch, in_ch, 1, stride=1, padding=0)
        self.proj = nn.Conv2d(in_ch, in_ch, 1, stride=1, padding=0)

    def forward(self, x):
        B, C, H, W = x.shape
        h = self.group_norm(x)
        q = self.proj_q(h)
        k = self.proj_k(h)
        v = self.proj_v(h)

        q = q.permute(0, 2, 3, 1).view(B, H * W, C)
        k = k.view(B, C, H * W)
        w = torch.bmm(q, k) * (int(C) ** (-0.5))
        assert list(w.shape) == [B, H * W, H * W]
        w = F.softmax(w, dim=-1)

        v = v.permute(0, 2, 3, 1).view(B, H * W, C)
        h = torch.bmm(w, v)
        assert list(h.shape) == [B, H * W, C]
        h = h.view(B, H, W, C).permute(0, 3, 1, 2)
        h = self.proj(h)

        return x + h



class ResBlock(nn.Module):
    def __init__(self, in_ch, out_ch, tdim, dropout, attn=True):
        super().__init__()
        self.block1 = nn.Sequential(
            nn.GroupNorm(24, in_ch),
            Swish(),
            nn.Conv2d(in_ch, out_ch, 3, stride=1, padding=1),
        )
        self.temb_proj = nn.Sequential(
            Swish(),
            nn.Linear(tdim, out_ch),
        )
        self.cond_proj = nn.Sequential(
            Swish(),
            nn.Linear(tdim, out_ch),
        )
        self.block2 = nn.Sequential(
            nn.GroupNorm(24, out_ch),
            Swish(),
            nn.Dropout(dropout),
            nn.Conv2d(out_ch, out_ch, 3, stride=1, padding=1),
        )
        if in_ch != out_ch:
            self.shortcut = nn.Conv2d(in_ch, out_ch, 1, stride=1, padding=0)
        else:
            self.shortcut = nn.Identity()
        if attn:
            self.attn = AttnBlock(out_ch)
        else:
            self.attn = nn.Identity()


    def forward(self, x, temb, labels):
        h = self.block1(x)
        h += self.temb_proj(temb)[:, :, None, None]
        h += self.cond_proj(labels)[:, :, None, None]
        h = self.block2(h)

        h = h + self.shortcut(x)
        h = self.attn(h)
        return h


class UNet(nn.Module):
    def __init__(self, T, num_labels, ch, ch_mult, num_res_blocks, dropout):
        super().__init__()
        tdim = ch * 4
        self.time_embedding = TimeEmbedding(T, ch, tdim)
        self.cond_embedding = ConditionalEmbedding(num_labels, ch, tdim)
        self.head = nn.Conv2d(3, ch, kernel_size=3, stride=1, padding=1)
        self.downblocks = nn.ModuleList()
        chs = [ch]  # record output channel when dowmsample for upsample
        now_ch = ch
        for i, mult in enumerate(ch_mult):
            out_ch = ch * mult
            for _ in range(num_res_blocks):
                self.downblocks.append(ResBlock(in_ch=now_ch, out_ch=out_ch, tdim=tdim, dropout=dropout))
                now_ch = out_ch
                chs.append(now_ch)
            if i != len(ch_mult) - 1:
                self.downblocks.append(DownSample(now_ch))
                chs.append(now_ch)

        self.middleblocks = nn.ModuleList([
            ResBlock(now_ch, now_ch, tdim, dropout, attn=True),
            ResBlock(now_ch, now_ch, tdim, dropout, attn=False),
        ])

        self.upblocks = nn.ModuleList()
        for i, mult in reversed(list(enumerate(ch_mult))):
            out_ch = ch * mult
            for _ in range(num_res_blocks + 1):
                self.upblocks.append(ResBlock(in_ch=chs.pop() + now_ch, out_ch=out_ch, tdim=tdim, dropout=dropout, attn=False))
                now_ch = out_ch
            if i != 0:
                self.upblocks.append(UpSample(now_ch))
        assert len(chs) == 0

        self.tail = nn.Sequential(
            nn.GroupNorm(32, now_ch),
            Swish(),
            nn.Conv2d(now_ch, 3, 3, stride=1, padding=1)
        )


    def forward(self, x, t, labels):
        # Timestep embedding
        temb = self.time_embedding(t)
        cemb = self.cond_embedding(labels)
        # Downsampling
        h = self.head(x)
        hs = [h]
        for layer in self.downblocks:
            h = layer(h, temb, cemb)
            hs.append(h)
        # Middle
        for layer in self.middleblocks:
            h = layer(h, temb, cemb)
        # Upsampling
        for layer in self.upblocks:
            if isinstance(layer, ResBlock):
                h = torch.cat([h, hs.pop()], dim=1)
            h = layer(h, temb, cemb)
        h = self.tail(h)

        assert len(hs) == 0
        return h


# if __name__ == '__main__':
#     batch_size = 8
#     model = UNet(
#         T=1000, num_labels=10, ch=128, ch_mult=[1, 2, 2, 2],
#         num_res_blocks=2, dropout=0.1)
#     x = torch.randn(batch_size, 3, 24, 24)
#     t = torch.randint(1000, size=[batch_size])
#     labels = torch.randint(10, size=[batch_size])
#     # resB = ResBlock(128, 256, 64, 0.1)
#     # x = torch.randn(batch_size, 128, 32, 32)
#     # t = torch.randn(batch_size, 64)
#     # labels = torch.randn(batch_size, 64)
#     # y = resB(x, t, labels)
#     y = model(x, t, labels)
#     print(y.shape)

In [2]:
# DiffusionCondition.py

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


def extract(v, t, x_shape):
    """
    Extract some coefficients at specified timesteps, then reshape to
    [batch_size, 1, 1, 1, 1, ...] for broadcasting purposes.
    """
    device = t.device
    out = torch.gather(v, index=t, dim=0).float().to(device)
    return out.view([t.shape[0]] + [1] * (len(x_shape) - 1))


class GaussianDiffusionTrainer(nn.Module):
    def __init__(self, model, beta_1, beta_T, T):
        super().__init__()

        self.model = model
        self.T = T

        self.register_buffer(
            'betas', torch.linspace(beta_1, beta_T, T).double())
        alphas = 1. - self.betas
        alphas_bar = torch.cumprod(alphas, dim=0)

        # calculations for diffusion q(x_t | x_{t-1}) and others
        self.register_buffer(
            'sqrt_alphas_bar', torch.sqrt(alphas_bar))
        self.register_buffer(
            'sqrt_one_minus_alphas_bar', torch.sqrt(1. - alphas_bar))

    def forward(self, x_0, labels):
        """
        Algorithm 1.
        """
        t = torch.randint(self.T, size=(x_0.shape[0], ), device=x_0.device)
        noise = torch.randn_like(x_0)
        x_t =   extract(self.sqrt_alphas_bar, t, x_0.shape) * x_0 + \
                extract(self.sqrt_one_minus_alphas_bar, t, x_0.shape) * noise
        loss = F.mse_loss(self.model(x_t, t, labels), noise, reduction='none')
        return loss


class GaussianDiffusionSampler(nn.Module):
    def __init__(self, model, beta_1, beta_T, T, w = 0.):
        super().__init__()

        self.model = model
        self.T = T
        ### In the classifier free guidence paper, w is the key to control the gudience.
        ### w = 0 and with label = 0 means no guidence.
        ### w > 0 and label > 0 means guidence. Guidence would be stronger if w is bigger.
        self.w = w

        self.register_buffer('betas', torch.linspace(beta_1, beta_T, T).double())
        alphas = 1. - self.betas
        alphas_bar = torch.cumprod(alphas, dim=0)
        alphas_bar_prev = F.pad(alphas_bar, [1, 0], value=1)[:T]
        self.register_buffer('coeff1', torch.sqrt(1. / alphas))
        self.register_buffer('coeff2', self.coeff1 * (1. - alphas) / torch.sqrt(1. - alphas_bar))
        self.register_buffer('posterior_var', self.betas * (1. - alphas_bar_prev) / (1. - alphas_bar))

    def predict_xt_prev_mean_from_eps(self, x_t, t, eps):
        assert x_t.shape == eps.shape
        return extract(self.coeff1, t, x_t.shape) * x_t - extract(self.coeff2, t, x_t.shape) * eps

    def p_mean_variance(self, x_t, t, labels):
        # below: only log_variance is used in the KL computations
        var = torch.cat([self.posterior_var[1:2], self.betas[1:]])
        var = extract(var, t, x_t.shape)
        eps = self.model(x_t, t, labels)
        nonEps = self.model(x_t, t, torch.zeros_like(labels).to(labels.device))
        eps = (1. + self.w) * eps - self.w * nonEps
        xt_prev_mean = self.predict_xt_prev_mean_from_eps(x_t, t, eps=eps)
        return xt_prev_mean, var

    def forward(self, x_T, labels):
        """
        Algorithm 2.
        """
        x_t = x_T
        for time_step in reversed(range(self.T)):
            print(time_step)
            t = x_t.new_ones([x_T.shape[0], ], dtype=torch.long) * time_step
            mean, var= self.p_mean_variance(x_t=x_t, t=t, labels=labels)
            if time_step > 0:
                noise = torch.randn_like(x_t)
            else:
                noise = 0
            x_t = mean + torch.sqrt(var) * noise
            assert torch.isnan(x_t).int().sum() == 0, "nan in tensor."
        x_0 = x_t
        return torch.clip(x_0, -1, 1)

In [6]:
# TrainCondition.py

import os
from typing import Dict
import numpy as np

import torch
import torch.optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import CIFAR10
from torchvision.utils import save_image

## new
import torchvision.transforms.functional as TF  # DCT
from scipy.fftpack import dct, idct
import numpy as np

def dct_batch(images: torch.Tensor) -> torch.Tensor:
    # Convert to numpy and apply DCT per channel
    images_np = images.cpu().numpy()
    B, C, H, W = images_np.shape
    dct_images = np.zeros_like(images_np)

    for b in range(B):
        for c in range(C):
            dct_images[b, c] = dct(dct(images_np[b, c], axis=0, norm='ortho'), axis=1, norm='ortho')
    compressed = dct_images[:, :, :24, :24]
    return torch.from_numpy(compressed).to(images.device)


def idct_batch(dct_images: torch.Tensor) -> torch.Tensor:
    # Convert to numpy and apply IDCT per channel
    dct_np = dct_images.cpu().numpy()
    B, C, H, W = dct_np.shape
    recon_images = np.zeros_like(dct_np)

    for b in range(B):
        for c in range(C):
            recon_images[b, c] = idct(idct(dct_np[b, c], axis=1, norm='ortho'), axis=0, norm='ortho')

    return torch.from_numpy(recon_images).to(dct_images.device)
##


def train(modelConfig: Dict):
    device = torch.device(modelConfig["device"])
    # dataset
    dataset = CIFAR10(
        root='./CIFAR10', train=True, download=True,
        transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]))
    dataloader = DataLoader(
        dataset, batch_size=modelConfig["batch_size"], shuffle=True, num_workers=4, drop_last=True, pin_memory=True)

    # model setup
    net_model = UNet(T=modelConfig["T"], num_labels=10, ch=modelConfig["channel"], ch_mult=modelConfig["channel_mult"],
                     num_res_blocks=modelConfig["num_res_blocks"], dropout=modelConfig["dropout"]).to(device)
    if modelConfig["training_load_weight"] is not None:
        net_model.load_state_dict(torch.load(os.path.join(
            modelConfig["save_dir"], modelConfig["training_load_weight"]), map_location=device), strict=False)
        print("Model weight load down.")
    optimizer = torch.optim.AdamW(
        net_model.parameters(), lr=modelConfig["lr"], weight_decay=1e-4)
    cosineScheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer=optimizer, T_max=modelConfig["epoch"], eta_min=0, last_epoch=-1)
    warmUpScheduler = GradualWarmupScheduler(optimizer=optimizer, multiplier=modelConfig["multiplier"],
                                             warm_epoch=modelConfig["epoch"] // 10, after_scheduler=cosineScheduler)
    trainer = GaussianDiffusionTrainer(
        net_model, modelConfig["beta_1"], modelConfig["beta_T"], modelConfig["T"]).to(device)

    losses = []  # new

    # start training
    losses = []
    lrs = []
    for e in range(modelConfig["epoch"]):
        epoch_losses = []
        epoch_lrs = []
        with tqdm(dataloader, dynamic_ncols=True) as tqdmDataLoader:
            for images_, labels in tqdmDataLoader:  # new var name
                # train
                images = dct_batch(images_)  # new
                b = images.shape[0]
                optimizer.zero_grad()
                x_0 = images.to(device)
                labels = labels.to(device) + 1
                if np.random.rand() < 0.1:
                    labels = torch.zeros_like(labels).to(device)
                loss = trainer(x_0, labels).sum() / b ** 2.
                loss.backward()
                torch.nn.utils.clip_grad_norm_(
                    net_model.parameters(), modelConfig["grad_clip"])
                optimizer.step()
                loss_item = loss.item()
                lr = optimizer.state_dict()['param_groups'][0]["lr"]
                epoch_lrs.append(lr)
                epoch_losses.append(loss_item)
                tqdmDataLoader.set_postfix(ordered_dict={
                    "epoch": e,
                    "loss: ": loss_item,
                    "img shape: ": x_0.shape,
                    "LR": lr
                })
            tqdmDataLoader.set_postfix(ordered_dict={
                "epoch": e,
                "loss: ": np.mean(epoch_losses),
                "img shape: ": x_0.shape,
                "LR": np.mean(epoch_lrs)
            })
        warmUpScheduler.step()
        losses.append(np.mean(epoch_losses))
        lrs.append(np.mean(epoch_lrs))
        torch.save(net_model.state_dict(), os.path.join(
            modelConfig["save_dir"], 'ckpt_' + str(e) + "_.pt"))
    print(losses)
    print(lrs)
    with open('./data.txt', 'w') as f:
        f.write(f"{losses}\n{lrs}")

    return losses  # new


def eval(modelConfig: Dict):
    device = torch.device(modelConfig["device"])
    # load model and evaluate
    with torch.no_grad():
        step = int(modelConfig["batch_size"] // 10)
        labelList = []
        k = 0
        for i in range(1, modelConfig["batch_size"] + 1):
            labelList.append(torch.ones(size=[1]).long() * k)
            if i % step == 0:
                if k < 10 - 1:
                    k += 1
        labels = torch.cat(labelList, dim=0).long().to(device) + 1
        print("labels: ", labels)
        model = UNet(T=modelConfig["T"], num_labels=10, ch=modelConfig["channel"], ch_mult=modelConfig["channel_mult"],
                     num_res_blocks=modelConfig["num_res_blocks"], dropout=modelConfig["dropout"]).to(device)
        print("path is", os.path.join(
            modelConfig["save_dir"], modelConfig["test_load_weight"]))
        ckpt = torch.load(os.path.join(
            modelConfig["save_dir"], modelConfig["test_load_weight"]), map_location=device)
        model.load_state_dict(ckpt)
        print("model load weight done.")
        model.eval()
        sampler = GaussianDiffusionSampler(
            model, modelConfig["beta_1"], modelConfig["beta_T"], modelConfig["T"], w=modelConfig["w"]).to(device)
        # Sampled from standard normal distribution
        noisyImage = torch.randn(
            size=[modelConfig["batch_size"], 3, modelConfig["img_size"], modelConfig["img_size"]], device=device)
        saveNoisy = torch.clamp(noisyImage * 0.5 + 0.5, 0, 1)
        save_image(saveNoisy, os.path.join(
            modelConfig["sampled_dir"],  modelConfig["sampledNoisyImgName"]), nrow=modelConfig["nrow"])
        sampledImgs = sampler(noisyImage, labels)
        sampledImgs = sampledImgs * 0.5 + 0.5  # [0 ~ 1]
        print(sampledImgs)
        save_image(sampledImgs, os.path.join(
            modelConfig["sampled_dir"],  modelConfig["sampledImgName"]), nrow=modelConfig["nrow"])

In [14]:
# main.py

def main(model_config=None):
    modelConfig = {
        "state": "eval", # or eval
        "epoch": 70,
        "batch_size": 80,
        "T": 500,
        "channel": 128,
        "channel_mult": [1, 2, 2, 2],
        "num_res_blocks": 2,
        "dropout": 0.15,
        "lr": 1e-4,
        "multiplier": 2.5,
        "beta_1": 1e-4,
        "beta_T": 0.028,
        "img_size": 32,
        "grad_clip": 1.,
        "device": "cuda:0",
        "w": 1.8,
        "save_dir": "./CheckpointsCondition",
        "training_load_weight": None,
        "test_load_weight": "../ckpt_69_fourier_new.pt",
        "sampled_dir": "./SampledImgs/",
        "sampledNoisyImgName": "NoisyGuidenceImgs.png",
        "sampledImgName": "SampledGuidenceImgs.png",
        "nrow": 8
    }
    if model_config is not None:
        modelConfig = model_config
    if modelConfig["state"] == "train":
        training_loss = train(modelConfig)  # new
        print(training_loss)  # new
    else:
        eval(modelConfig)


if __name__ == '__main__':
    main()

labels:  tensor([ 1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,
         3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,
         5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,
         7,  7,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10], device='cuda:0')
path is ./CheckpointsCondition/../ckpt_69_fourier_new.pt


<ipython-input-11-6bcaf096ff2d>:145: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(os.path.join(


model load weight done.
499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
399
398
397
396
395
394
393
392
391
390
389
388
387
386
385
384
383
382
381
380
379
378
377
376
375
374
373
372
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
333
332
331
330
329
328
327
326
325
324
323
322
321
320
319
318
317
316
315
314
313
312
311
310
309
308
307
306
305
304
303
302
301
300
299
298
297
296
295
294
293
292
291
290
289
288
287
286
285
284
283
282
281
280
279
278
277
276
275
274
273
272
271
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256


KeyboardInterrupt: 

In [4]:
%pip install lpips

In [7]:
from ast import NodeTransformer
import os
from typing import Dict

import torch
import numpy as np
from scipy import linalg, stats
%pip install lpips
import lpips

from torchvision import transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, Subset
from torchvision.models import inception_v3, Inception_V3_Weights
from torch.nn import functional as F
from torchvision.utils import save_image

# ──────────────────────────────────────────────────────────────────────────────
# 1. METRIC FUNCTIONS
# ──────────────────────────────────────────────────────────────────────────────

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load models
weights = Inception_V3_Weights.DEFAULT
_inception = inception_v3(weights=weights, aux_logits=True).to(device).eval()
# and grab its recommended preprocessing transform:
_incep_preprocess = weights.transforms()

# _lpips = lpips.LPIPS(net='alex').to(device)
_lpips = lpips.LPIPS(net='squeeze').to(device)

def _get_inception_logits(x: torch.Tensor) -> torch.Tensor:
    x_proc = _incep_preprocess(x)           # resize → crop → normalize
    with torch.no_grad():
      out = _inception(x_proc.to(device))
    if isinstance(out, tuple):
        out = out[0]                        # drop the aux_logits
    return out

def calculate_fid(mu1, sigma1, mu2, sigma2):
    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    covmean = covmean.real if np.iscomplexobj(covmean) else covmean
    return diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean)

def compute_fid(real: torch.Tensor, fake: torch.Tensor) -> float:
    act_real = _get_inception_logits(real).cpu().numpy()
    act_fake = _get_inception_logits(fake).cpu().numpy()
    mu_r, sigma_r = act_real.mean(0), np.cov(act_real, rowvar=False)
    mu_f, sigma_f = act_fake.mean(0), np.cov(act_fake, rowvar=False)
    return calculate_fid(mu_r, sigma_r, mu_f, sigma_f)

def compute_inception_score(fake: torch.Tensor, splits=10):
    preds = F.softmax(_get_inception_logits(fake), dim=1).cpu().numpy()
    N = preds.shape[0]
    scores = []
    for k in range(splits):
        part = preds[k*(N//splits):(k+1)*(N//splits)]
        py = part.mean(axis=0)
        scores.append(np.exp(np.mean([stats.entropy(p, py) for p in part])))
    return float(np.mean(scores)), float(np.std(scores))

def compute_lpips(real: torch.Tensor, fake: torch.Tensor) -> float:
    # LPIPS expects input in [-1,+1]
    r = 2*real - 1
    f = 2*fake - 1
    with torch.no_grad():
        return float(_lpips(f.to(device), r.to(device)).mean())

def evaluate_all(real: torch.Tensor, fake: torch.Tensor) -> Dict[str, float]:
    return {
        "FID": compute_fid(real, fake),
        "IS_mean": compute_inception_score(fake)[0],
        "IS_std":  compute_inception_score(fake)[1],
        "LPIPS": compute_lpips(real, fake),
    }

# ──────────────────────────────────────────────────────────────────────────────
# 2. REAL CIFAR-10 LOADER
# ──────────────────────────────────────────────────────────────────────────────

def load_cifar10_test(num_samples: int):
    """
    Returns:
      real_images: Tensor [num_samples,3,32,32] in [0,1]
      real_labels: Tensor [num_samples]
    """
    ds = CIFAR10(root="./data", train=False, download=True,
                 transform=transforms.ToTensor())
    if num_samples < len(ds):
        ds = Subset(ds, list(range(num_samples)))
    loader = DataLoader(ds, batch_size=num_samples, shuffle=False)
    _imgs, labs = next(iter(loader))
    imgs = dct_batch(_imgs)  # compute DCT of images before
    return imgs.to(device), labs.to(device)

# ──────────────────────────────────────────────────────────────────────────────
# 3. EXTENDED eval() WITH METRICS
# ──────────────────────────────────────────────────────────────────────────────

def eval_with_metrics(modelConfig: Dict):
    device = torch.device(modelConfig["device"])
    B = modelConfig["batch_size"]

    # --- load real CIFAR-10 test images
    real_images, real_labels = load_cifar10_test(num_samples=B)
    print(f"Loaded {real_images.shape[0]} real CIFAR-10 test images.")

    with torch.no_grad():
        # build label tensor [0..9] repeated
        step = B // 10
        label_list = []
        for k in range(10):
            label_list += [torch.full((step,), k, dtype=torch.long)]
        labels = torch.cat(label_list, dim=0).to(device)
        print("Labels tensor:", labels.shape)

        # instantiate and load weights
        model = UNet(
            T=modelConfig["T"],
            num_labels=10,
            ch=modelConfig["channel"],
            ch_mult=modelConfig["channel_mult"],
            num_res_blocks=modelConfig["num_res_blocks"],
            dropout=modelConfig["dropout"]
        ).to(device)
        ckpt = torch.load(
            os.path.join(modelConfig["save_dir"], modelConfig["test_load_weight"]),
            map_location=device
        )
        model.load_state_dict(ckpt)
        model.eval()
        print("Model loaded.")

        # sampler
        sampler = GaussianDiffusionSampler(
            model,
            modelConfig["beta_1"],
            modelConfig["beta_T"],
            modelConfig["T"],
            w=modelConfig["w"]
        ).to(device)

        # sample from N(0,I)
        noisy = torch.randn(
            B, 3, modelConfig["img_size"], modelConfig["img_size"],
            device=device
        )
        sampled = sampler(noisy, labels)
        sampled = sampled.clamp(-0.5, +0.5) + 0.5   # to [0,1]
        print(f"Generated {sampled.shape[0]} fake images.")

        # (optional) save a grid
        save_image(
            sampled,
            os.path.join(modelConfig["sampled_dir"], "SampledGuidanceImgs_fourier_dct.png"),
            nrow=modelConfig["nrow"]
        )
        back_to_normal = idct_batch(sampled)
        save_image(
            back_to_normal,
            os.path.join(modelConfig["sampled_dir"], "SampledGuidanceImgs_fourier_nodct.png"),
            nrow=modelConfig["nrow"]
        )
        norm_to_normal = back_to_normal * 0.5 + 0.5
        save_image(
            norm_to_normal,
            os.path.join(modelConfig["sampled_dir"], "SampledGuidanceImgs_fourier_nodct_norm.png"),
            nrow=modelConfig["nrow"]
        )
        # backnforth = idct_batch(dct_batch(real_images))* 0.5 + 0.5
        # save_image(
        #     backnforth,
        #     os.path.join(modelConfig["sampled_dir"], "SampledGuidanceImgs_fourier_DCTandinv.png"),
        #     nrow=modelConfig["nrow"]
        # )


    # --- compute metrics
    metrics = evaluate_all(real_images, sampled)
    print("=== Evaluation Metrics ===")
    for k,v in metrics.items():
        print(f"{k}: {v:.4f}")

    return metrics


def main(model_config=None):
    modelConfig = {
        "state": "eval", # or eval
        "epoch": 70,
        "batch_size": 80,
        "T": 500,
        "channel": 96,
        "channel_mult": [1, 2, 2, 2],
        "num_res_blocks": 2,
        "dropout": 0.15,
        "lr": 1e-4,
        "multiplier": 2.5,
        "beta_1": 1e-4,
        "beta_T": 0.028,
        "img_size": 24,
        "grad_clip": 1.,
        "device": "cuda:0",
        "w": 1.8,
        "save_dir": "./",
        "training_load_weight": None,
        "test_load_weight": "ckpt_19_.pt",
        "sampled_dir": "./SampledImgs/",
        "sampledNoisyImgName": "NoisyGuidenceImgs.png",
        "sampledImgName": "SampledGuidenceImgs.png",
        "nrow": 8
    }
    if model_config is not None:
        modelConfig = model_config
    if modelConfig["state"] == "train":
        training_loss = train(modelConfig)  # new
        print(training_loss)  # new
    else:
        metrics = eval_with_metrics(modelConfig)
        print(metrics)

main()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_1_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_1_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_1-b8a52dc0.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_1-b8a52dc0.pth


Setting up [LPIPS] perceptual loss: trunk [squeeze], v[0.1], spatial [off]


100%|██████████| 4.73M/4.73M [00:00<00:00, 22.6MB/s]
/usr/local/lib/python3.10/dist-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_stat

Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/squeeze.pth
Files already downloaded and verified
Loaded 80 real CIFAR-10 test images.
Labels tensor: torch.Size([80])


<ipython-input-7-8b319514d720>:128: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(


Model loaded.
499
498
497
496
495
494
493
492
491
490
489
488
487
486
485
484
483
482
481
480
479
478
477
476
475
474
473
472
471
470
469
468
467
466
465
464
463
462
461
460
459
458
457
456
455
454
453
452
451
450
449
448
447
446
445
444
443
442
441
440
439
438
437
436
435
434
433
432
431
430
429
428
427
426
425
424
423
422
421
420
419
418
417
416
415
414
413
412
411
410
409
408
407
406
405
404
403
402
401
400
399
398
397
396
395
394
393
392
391
390
389
388
387
386
385
384
383
382
381
380
379
378
377
376
375
374
373
372
371
370
369
368
367
366
365
364
363
362
361
360
359
358
357
356
355
354
353
352
351
350
349
348
347
346
345
344
343
342
341
340
339
338
337
336
335
334
333
332
331
330
329
328
327
326
325
324
323
322
321
320
319
318
317
316
315
314
313
312
311
310
309
308
307
306
305
304
303
302
301
300
299
298
297
296
295
294
293
292
291
290
289
288
287
286
285
284
283
282
281
280
279
278
277
276
275
274
273
272
271
270
269
268
267
266
265
264
263
262
261
260
259
258
257
256
255
254
25

In [ ]:
##### SANITY CHECK FOR IMAGE
def load_cifar10_testt(num_samples: int):
    """
    Returns:
      real_images: Tensor [num_samples,3,32,32] in [0,1]
      real_labels: Tensor [num_samples]
    """
    ds = CIFAR10(root="./data", train=False, download=True,
                 transform=transforms.ToTensor())
    if num_samples < len(ds):
        ds = Subset(ds, list(range(num_samples)))
    loader = DataLoader(ds, batch_size=num_samples, shuffle=False)
    _imgs, labs = next(iter(loader))
    # imgs = dct_batch(_imgs)  # compute DCT of images before
    return _imgs.to(device), labs.to(device)
def sanity_check(modelConfig: Dict):
    device = torch.device(modelConfig["device"])
    B = modelConfig["batch_size"]

    # --- load real CIFAR-10 test images
    real_images, real_labels = load_cifar10_testt(num_samples=B)
    save_image(
        real_images *0.5 + 0.5,
        os.path.join("./BASELINE_1.png"),
        nrow=modelConfig["nrow"]
    )
    backnforth = idct_batch(dct_batch(real_images))* 0.5 + 0.5
    backnforth_no_clamp = idct_batch(dct_batch(real_images))

    save_image(
        backnforth,
        os.path.join("./SampledGuidanceImgs_fourier_DCTandinv.png"),
        nrow=modelConfig["nrow"]
    )
    save_image(
        backnforth,
        os.path.join("./SampledGuidanceImgs_fourier_DCTandinv_no_clamp.png"),
        nrow=modelConfig["nrow"]
    )
    save_image(
        real_images,
        os.path.join("./BASELINE.png"),
        nrow=modelConfig["nrow"]
    )
modelConfig = {
        "state": "eval", # or eval
        "epoch": 70,
        "batch_size": 80,
        "T": 500,
        "channel": 128,
        "channel_mult": [1, 2, 2, 2],
        "num_res_blocks": 2,
        "dropout": 0.15,
        "lr": 1e-4,
        "multiplier": 2.5,
        "beta_1": 1e-4,
        "beta_T": 0.028,
        "img_size": 32,
        "grad_clip": 1.,
        "device": "cuda:0",
        "w": 1.8,
        "save_dir": "./CheckpointsCondition2/",
        "training_load_weight": None,
        "test_load_weight": "ckpt_69_.pt",
        "sampled_dir": "./SampledImgs/",
        "sampledNoisyImgName": "NoisyGuidenceImgs.png",
        "sampledImgName": "SampledGuidenceImgs.png",
        "nrow": 8
    }
sanity_check(modelConfig)

Files already downloaded and verified
